<a href="https://colab.research.google.com/github/gilcesarf/imd3002-202502/blob/main/Aula05/Exercicios_Aula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup inicial

In [15]:
import numpy as np
import pandas as pd

# Download the raw CSV data directly from GitHub

url = "https://raw.githubusercontent.com/gilcesarf/imd3002-202502/refs/heads/main/Aula05/data/Adult.csv"

dataset = pd.read_csv(url, encoding='utf-8')

# Características da Base de Dados


In [16]:
# caracteristicas da base de dados

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16281 non-null  int64 
 1   workclass       15318 non-null  object
 2   fnlwgt          16281 non-null  int64 
 3   education       16280 non-null  object
 4   education-num   16281 non-null  int64 
 5   marital-status  16281 non-null  object
 6   occupation      15315 non-null  object
 7   relationship    16281 non-null  object
 8   race            16281 non-null  object
 9   sex             16281 non-null  object
 10  capital-gain    16281 non-null  int64 
 11  capital-loss    16281 non-null  int64 
 12  hours-per-week  16281 non-null  int64 
 13  native-country  16007 non-null  object
 14  class           16281 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


# Verificação de Missing Values


In [17]:
# verificacao de missing values

# Transformação de Atributos Numéricos

In [18]:
# transformacao de atributos numericos

# Transformação de Atributos Discretos

In [19]:
# transformação de atributos discretos